In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [22]:
df = spark.read.csv("minute_weather.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)



In [23]:
df =df[df['_c0']<10000]

In [24]:
df.count()

10000

In [25]:
df = df.drop('_c0')
df = df.drop('_c1')
df = df.drop('_c2')
df = df.drop('_c5')
df = df.drop('_c7')
df = df.drop('_c9')
df = df.drop('_c10')
df = df.drop('_c11')
df.show()

+-----+-----+-----+-----+----+
|  _c3|  _c4|  _c6|  _c8|_c12|
+-----+-----+-----+-----+----+
|64.76| 97.0|106.0| 85.0|60.5|
|63.86|161.0|215.0| 43.0|39.9|
|64.22| 77.0|143.0|324.0|43.0|
| 64.4| 89.0|112.0| 12.0|49.5|
| 64.4|185.0|260.0|100.0|58.8|
| 63.5| 76.0| 92.0| 61.0|62.6|
|62.78| 79.0| 89.0| 62.0|65.6|
|62.42| 86.0| 92.0| 75.0|65.2|
|62.24|105.0|125.0| 82.0|65.8|
|62.24| 93.0|126.0| 14.0|58.6|
|62.24|144.0|167.0|115.0|38.5|
|63.14|105.0|126.0| 92.0|42.6|
|64.04|116.0|143.0|104.0|45.3|
| 64.4|142.0|200.0| 93.0|36.1|
|64.94|150.0|173.0|117.0|33.2|
|65.48| 90.0|100.0| 80.0|45.2|
|65.48| 91.0|168.0| 26.0|55.9|
|64.94| 77.0| 91.0| 55.0|60.5|
|64.04|107.0|116.0| 95.0|61.8|
|63.32|109.0|119.0| 98.0|56.0|
+-----+-----+-----+-----+----+
only showing top 20 rows



In [26]:
df=df.na.drop()

In [27]:
df.count()

10000

In [28]:
from pyspark.sql.functions import col , column
df = df.withColumn("_c3", col("_c3").cast("double"))
df = df.withColumn("_c4", col("_c4").cast("double"))
df = df.withColumn("_c6", col("_c6").cast("double"))
df = df.withColumn("_c8", col("_c8").cast("double"))
df = df.withColumn("_c12", col("_c12").cast("double"))

In [29]:
# random vi tri bat dau
def random_init(max_min,k,len_columns):
    import random
    k_position=[]
    import random
    for i in range(0,k):
        row=[]
        for j in range(0,len_columns):
            row.append( random.randrange(int(max_min[1][j]),int(max_min[0][j])))
        k_position.append(row)
    return k_position
def find_closeness(position,k_position):
    min_closness = 0
    index = 0
    for i in range (0 , len(k_position)):
        distance = 0
        for j in range (0,len(position)):
            distance = distance + (position[j]-k_position[i][j])**2
        if i == 0:
            min_closness = distance
        else:
            if distance < min_closness:
                min_closness = distance
                index = i
    return index


def update_position(df,k,name_columns):
    data = df.groupBy('group').avg().collect()
    k_position =[]
    lenght_columns = len(name_columns)
    for i in range (0,k):
        index = [] 
        for j in range (0,lenght_columns):
            name = 'avg(' + str(name_columns[j]) +')'
            index.append(data[i][name])
        k_position.append(index)
    return k_position
    
def k_mean(df,k):
    from pyspark.sql.functions import lit
    from pyspark.sql import Row
    import pyspark.sql.functions as F     

    def classtify(row):
        rd=row.asDict()
        len_columns=len(k_position[0])
        position=[]
        for i in range (0,len_columns):
            position.append(float(row[i]))
        index = find_closeness(position,k_position)
        if row['group'] == index:
            rd["status"]=0
        else:
            rd["status"]=1
        rd["group"]=index
        new_row=Row(**rd)  
        return new_row


    columns = df.columns
    len_columns = len(columns)
    max_value=[]
    min_value=[]
    max_min = []
    for i in  columns:
        row_max = df.agg({i: "max"}).collect()[0]
        name_column = 'max('+i+')'
        max_value.append(row_max[name_column])
        
        row_min = df.agg({i: "min"}).collect()[0]
        name_column = 'min('+i+')'
        min_value.append(row_min[name_column])
    max_min.append(max_value)
    max_min.append(min_value)
    
#     luu gia tri khoi tao vi tri dau
    k_position =  random_init(max_min,k,len_columns)
#     luu vi tri gan nhay
#     them column group
    df=df.withColumn('group', lit(-1))
#     phan lop
    while True:
        df=df.rdd.map(classtify).toDF()
        k_position = update_position(df,k,columns)
        if df.agg(F.sum("status")).collect()[0][0] == 0 :
            break
    return df.drop('status')
df = k_mean(df,3)


In [30]:
df.collect()

[Row(_c3=64.76, _c4=97.0, _c6=106.0, _c8=85.0, _c12=60.5, group=1),
 Row(_c3=63.86, _c4=161.0, _c6=215.0, _c8=43.0, _c12=39.9, group=0),
 Row(_c3=64.22, _c4=77.0, _c6=143.0, _c8=324.0, _c12=43.0, group=2),
 Row(_c3=64.4, _c4=89.0, _c6=112.0, _c8=12.0, _c12=49.5, group=1),
 Row(_c3=64.4, _c4=185.0, _c6=260.0, _c8=100.0, _c12=58.8, group=0),
 Row(_c3=63.5, _c4=76.0, _c6=92.0, _c8=61.0, _c12=62.6, group=1),
 Row(_c3=62.78, _c4=79.0, _c6=89.0, _c8=62.0, _c12=65.6, group=1),
 Row(_c3=62.42, _c4=86.0, _c6=92.0, _c8=75.0, _c12=65.2, group=1),
 Row(_c3=62.24, _c4=105.0, _c6=125.0, _c8=82.0, _c12=65.8, group=1),
 Row(_c3=62.24, _c4=93.0, _c6=126.0, _c8=14.0, _c12=58.6, group=1),
 Row(_c3=62.24, _c4=144.0, _c6=167.0, _c8=115.0, _c12=38.5, group=0),
 Row(_c3=63.14, _c4=105.0, _c6=126.0, _c8=92.0, _c12=42.6, group=1),
 Row(_c3=64.04, _c4=116.0, _c6=143.0, _c8=104.0, _c12=45.3, group=1),
 Row(_c3=64.4, _c4=142.0, _c6=200.0, _c8=93.0, _c12=36.1, group=0),
 Row(_c3=64.94, _c4=150.0, _c6=173.0, _c8=11

In [32]:
df[df['group']==0].show()
df[df['group']==0].count()

+-----+-----+-----+-----+----+-----+
|  _c3|  _c4|  _c6|  _c8|_c12|group|
+-----+-----+-----+-----+----+-----+
|63.86|161.0|215.0| 43.0|39.9|    0|
| 64.4|185.0|260.0|100.0|58.8|    0|
|62.24|144.0|167.0|115.0|38.5|    0|
| 64.4|142.0|200.0| 93.0|36.1|    0|
|64.94|150.0|173.0|117.0|33.2|    0|
|63.32|144.0|200.0| 75.0|54.5|    0|
| 64.4|156.0|203.0|119.0|50.4|    0|
| 64.4|147.0|173.0|125.0|45.2|    0|
|64.76|152.0|171.0|116.0|45.6|    0|
|65.12|174.0|202.0|156.0|46.2|    0|
|65.48|162.0|203.0|127.0|45.2|    0|
|64.58|154.0|176.0|113.0|50.2|    0|
|64.76|149.0|188.0|123.0|46.8|    0|
|64.94|162.0|179.0|148.0|47.5|    0|
| 65.3|152.0|174.0|137.0|45.5|    0|
|65.48|170.0|220.0|126.0|45.4|    0|
|65.66|170.0|184.0|158.0|47.9|    0|
|65.66|170.0|227.0|153.0|49.8|    0|
|65.48|144.0|166.0|116.0|47.4|    0|
|65.48|159.0|185.0|111.0|46.8|    0|
+-----+-----+-----+-----+----+-----+
only showing top 20 rows



6530

In [33]:
df[df['group']==1].show()
df[df['group']==1].count()

+-----+-----+-----+-----+----+-----+
|  _c3|  _c4|  _c6|  _c8|_c12|group|
+-----+-----+-----+-----+----+-----+
|64.76| 97.0|106.0| 85.0|60.5|    1|
| 64.4| 89.0|112.0| 12.0|49.5|    1|
| 63.5| 76.0| 92.0| 61.0|62.6|    1|
|62.78| 79.0| 89.0| 62.0|65.6|    1|
|62.42| 86.0| 92.0| 75.0|65.2|    1|
|62.24|105.0|125.0| 82.0|65.8|    1|
|62.24| 93.0|126.0| 14.0|58.6|    1|
|63.14|105.0|126.0| 92.0|42.6|    1|
|64.04|116.0|143.0|104.0|45.3|    1|
|65.48| 90.0|100.0| 80.0|45.2|    1|
|65.48| 91.0|168.0| 26.0|55.9|    1|
|64.94| 77.0| 91.0| 55.0|60.5|    1|
|64.04|107.0|116.0| 95.0|61.8|    1|
|63.32|109.0|119.0| 98.0|56.0|    1|
|63.32|100.0|122.0| 91.0|58.3|    1|
|63.32|101.0|107.0| 90.0|55.1|    1|
|63.32| 96.0|119.0| 87.0|57.5|    1|
| 63.5|103.0|134.0| 79.0|45.7|    1|
|63.86| 85.0| 88.0| 81.0|61.5|    1|
|63.68| 85.0|103.0| 70.0|62.9|    1|
+-----+-----+-----+-----+----+-----+
only showing top 20 rows



3013

In [34]:
df[df['group']==2].show()
df[df['group']==2].count()

+-----+-----+-----+-----+----+-----+
|  _c3|  _c4|  _c6|  _c8|_c12|group|
+-----+-----+-----+-----+----+-----+
|64.22| 77.0|143.0|324.0|43.0|    2|
|65.66| 88.0|128.0|347.0|44.9|    2|
|67.82|  4.0| 25.0|348.0|34.2|    2|
| 67.1| 94.0|135.0|314.0|40.3|    2|
|66.56| 39.0|145.0|343.0|41.6|    2|
| 66.2| 44.0| 86.0|338.0|42.4|    2|
|64.58| 43.0|156.0|326.0|43.2|    2|
|52.88| 17.0| 36.0|323.0|87.3|    2|
|52.88| 69.0|142.0|358.0|87.6|    2|
|53.24|  6.0| 53.0|343.0|89.1|    2|
|53.06| 29.0| 56.0|348.0|89.1|    2|
|53.24| 18.0|102.0|234.0|90.4|    2|
|52.88|113.0|192.0|333.0|90.2|    2|
|53.06|  3.0| 23.0|344.0|90.4|    2|
|53.24| 10.0| 26.0|356.0|90.6|    2|
|53.42|  6.0| 16.0|357.0|90.8|    2|
|53.78| 18.0| 36.0|358.0|90.2|    2|
|54.32| 29.0| 99.0|277.0|90.0|    2|
| 54.5| 34.0| 82.0|315.0|90.0|    2|
| 54.5| 16.0| 50.0|345.0|89.6|    2|
+-----+-----+-----+-----+----+-----+
only showing top 20 rows



457